In [1]:
#Load the dataset
import pandas as pd

dataset = pd.read_csv('file9.csv')
dataset

,id,label,English
0,0,0,: Secular State Maharashtra:\n\n1. Ticktock St...
1,1,0,: Secular State Maharashtra:\n\n1. Ticktock St...
2,2,0,": Today, March 5, Friday at 6.15 pm All the pr..."
3,3,1,: Arrived at IDH Jaipur (SMS Hospital) here an...
4,4,1,"Our demands are illegitimate,\nReject our appl..."
5,5,0,African and Caribbean countries support India ...
6,6,0,: Secular State Maharashtra:\n\n1. Ticktock St...
7,7,0,": In the Kovid-19 Lump Sum Solution Scheme, do..."
8,8,1,Sub-divisional magistrate Sandeep Kumar said t...
9,9,0,: Arrived at IDH Jaipur (SMS Hospital) here an...


In [2]:

for index, tweet in enumerate(dataset["English"][10:15]):
    print(index+1,".",tweet)

1 . : Vaccinated it. Also, to make the process of vaccination smooth and smooth, the entire team of SMS Hospital including Dr. Sudhir Bhandari…
2 . : Jaipur
covidvacination
Former CM Vasundhara Raje took
First dose of Kovid-19 vaccine
Today at the IDH Center of SMS Hospital…
3 . : Arrived at IDH Jaipur (SMS Hospital) here and got the first dose of Kovid vaccine. Don't be afraid of the Kovid vaccine, it's safe…
4 . No secret, top US epidemiologists attribute 9 actions of the government to the decline in Kovid cases in India.
https://t.co/BvsZFULekb

via NaMo App https://t.co/FoTDSmdUdr
5 . : Secular State Maharashtra:

1. Ticktock Star dies suspiciously because of a Cabinet Minister.

2. In the women's hostel of Jalgaon…


In [3]:

import re

#Clean text from noise
def clean_text(text):
    #Filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    #Convert to lowercase to maintain consistency
    text = text.lower()
       
    return text

In [4]:
dataset['clean_text'] = dataset.English.apply(lambda x: clean_text(x))

In [5]:

STOP_WORDS = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'also', 'am', 'an', 'and',
              'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below',
              'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'com', 'could', "couldn't", 'did',
              "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'else', 'ever',
              'few', 'for', 'from', 'further', 'get', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having',
              'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how',
              "how's", 'however', 'http', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it',
              "it's", 'its', 'itself', 'just', 'k', "let's", 'like', 'me', 'more', 'most', "mustn't", 'my', 'myself',
              'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'otherwise', 'ought', 'our', 'ours',
              'ourselves', 'out', 'over', 'own', 'r', 'same', 'shall', "shan't", 'she', "she'd", "she'll", "she's",
              'should', "shouldn't", 'since', 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs',
              'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", "they'll", "they're",
              "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', "wasn't",
              'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
              "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would', "wouldn't",
              'www', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']

#Generate word frequency
def gen_freq(text):
    #Will store the list of words
    word_list = []

    #Loop over all the tweets and extract words into word_list
    for tw_words in text.split():
        word_list.extend(tw_words)

    #Create word frequencies using word_list
    word_freq = pd.Series(word_list).value_counts()
    
    #Drop the stopwords during the frequency calculation
    word_freq = word_freq.drop(STOP_WORDS, errors='ignore')
    
    return word_freq

#Check whether a negation term is present in the text
def any_neg(words):
    for word in words:
        if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
            return 1
    else:
        return 0

#Check whether one of the 100 rare words is present in the text
def any_rare(words, rare_100):
    for word in words:
        if word in rare_100:
            return 1
    else:
        return 0

#Check whether prompt words are present
def is_question(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who']:
            return 1
    else:
        return 0

In [6]:

word_freq = gen_freq(dataset.clean_text.str)
#100 most rare words in the dataset
rare_100 = word_freq[-100:]
#Number of words in a tweet
dataset['word_count'] = dataset.clean_text.str.split().apply(lambda x: len(x))
#Negation present or not
dataset['any_neg'] = dataset.clean_text.str.split().apply(lambda x: any_neg(x))
#Prompt present or not
dataset['is_question'] = dataset.clean_text.str.split().apply(lambda x: is_question(x))
#Any of the most 100 rare words present or not
dataset['any_rare'] = dataset.clean_text.str.split().apply(lambda x: any_rare(x, rare_100))
#Character count of the tweet
dataset['char_count'] = dataset.clean_text.apply(lambda x: len(x))

In [7]:

#Top 10 common words are
gen_freq(dataset.clean_text.str)[:10]

kovid       13
vaccine     11
sms          5
hospital     5
co           4
ticktock     4
jaipur       4
hostel       4
state        4
safe         4
dtype: int64

In [8]:
dataset

,id,label,English,clean_text,word_count,any_neg,is_question,any_rare,char_count
0,0,0,: Secular State Maharashtra:\n\n1. Ticktock St...,secular state maharashtra ticktock star...,18,0,0,0,133
1,1,0,: Secular State Maharashtra:\n\n1. Ticktock St...,secular state maharashtra ticktock star...,18,0,0,0,133
2,2,0,": Today, March 5, Friday at 6.15 pm All the pr...",today march friday at pm all the pr...,20,0,0,1,124
3,3,1,: Arrived at IDH Jaipur (SMS Hospital) here an...,arrived at idh jaipur sms hospital here an...,24,1,0,1,133
4,4,1,"Our demands are illegitimate,\nReject our appl...",our demands are illegitimate reject our appli...,30,1,0,1,191
5,5,0,African and Caribbean countries support India ...,african and caribbean countries support india ...,24,0,0,1,141
6,6,0,: Secular State Maharashtra:\n\n1. Ticktock St...,secular state maharashtra ticktock star...,18,0,0,0,133
7,7,0,": In the Kovid-19 Lump Sum Solution Scheme, do...",in the kovid lump sum solution scheme do...,22,0,0,1,146
8,8,1,Sub-divisional magistrate Sandeep Kumar said t...,sub divisional magistrate sandeep kumar said t...,37,1,1,1,226
9,9,0,: Arrived at IDH Jaipur (SMS Hospital) here an...,arrived at idh jaipur sms hospital here an...,24,1,0,1,133


In [9]:

from sklearn.model_selection import train_test_split

X = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
y = dataset.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=27)

In [10]:

from sklearn.naive_bayes import GaussianNB

#Initialize GaussianNB classifier
model = GaussianNB()
#Fit the model on the train dataset
model = model.fit(X_train, y_train)
#Make predictions on the test dataset
pred = model.predict(X_test)

In [11]:
from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_test, pred)*100, "%")

Accuracy: 100.0 %
